# core

> This is where users start to interact with the underlying LLM and agents.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
# install dependencies
# %pip install replicate
# %pip install gradio
# %pip install langchain

In [ ]:
#| export
# import replicate
# import gradio
# import langchain

import os
from getpass import getpass

# Enter REPLICATE API TOKEN to run inference
REPLICATE_API_TOKEN = getpass(prompt="Enter REPLICATE API TOKEN: ")
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

In [ ]:
#| export
import replicate

# function to run inference with Meta's Llama 3.8B model
def llama3_8b(prompt):
    output = replicate.run(
      "meta/meta-llama-3-8b-instruct",
      input={"prompt": prompt}
    )
    return ''.join(output)

In [ ]:
#| export
# A chatbot that allows users to interact with an LLM
import gradio as gr

def taskRouter(message, history):
    if message.endswith("?"):
        return "Good question!"
    elif "Hello" in message:
        return "Hello! How can I help?"
    else:
        return "Let me work on that...."

def vote(data: gr.LikeData):
    if data.liked:
        print("You upvoted this response: " + data.value["value"])
    else:
        print("You downvoted this response: " + data.value["value"])

with gr.Blocks() as sciChat:
    chatbot = gr.Chatbot(height=300,placeholder="<strong>Your Personal Science AI Assistant</strong><br>Ask Me Anything")
    chatbot.like(vote, None, None)
    gr.ChatInterface(
        fn=taskRouter,
        type="messages",
        chatbot=chatbot,
        textbox=gr.MultimodalTextbox(placeholder="How can I help?", container=False, scale=7),
        title="SciAI Assistant",
        description="Ask me any question or carry out a task",
        theme="soft",
        examples=[{"text": "Hello", "files": []},{"text": "What do you see in this image?", "files": []}],
        cache_examples=True,
        multimodal=True
    )
    
sciChat.launch(share=True)

/Users/danc/anaconda3/envs/llm/lib/python3.12/site-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(
/Users/danc/anaconda3/envs/llm/lib/python3.12/site-packages/gradio/chat_interface.py:221: UserWarning: The type of the chatbot does not match the type of the chat interface. The type of the chat interface will be used.Recieved type of chatbot: {chatbot.type}, type of chat interface: {self.type}
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
Using cache from '/Volumes/SSD2TB/Google Drive/Work/MachineLearning/SciAI/nbs/.gradio/cached_examples/11' directory. If method or examples have changed since last caching, delete this folder to clear cache.


Could not create share link. Missing file: /Users/danc/anaconda3/envs/llm/lib/python3.12/site-packages/gradio/frpc_darwin_amd64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_darwin_amd64
2. Rename the downloaded file to: frpc_darwin_amd64_v0.3
3. Move the file to this location: /Users/danc/anaconda3/envs/llm/lib/python3.12/site-packages/gradio


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()